In [11]:
import numpy as np
import pandas as pd

from tqdm import trange
import re
import joblib

In [3]:
test = pd.read_csv("test.csv")
val = pd.read_csv("val.csv")
train = pd.read_csv("Top1Million_and_generate.csv")

In [4]:
test.head()

,domain
0,dxveuu-km.ru
1,bellkrolik.ru
2,livetvstatic.ru
3,qyn73rku0zr.ru
4,ipvanish.com


In [6]:
val.head()

,domain,is_dga
0,r8s3-zzdxp.ru,1
1,mysolarfocus.com,0
2,m60oax.ru,1
3,kbgckwrax.nl,1
4,mostbetru32new.ru,0


In [7]:
train.head()

,domain,is_dga
0,freeslotsland.eu,0
1,digianime.co.jp,0
2,ionicacademy.com,0
3,bgnul1,1
4,fmoviesub.com,0


In [8]:
len(test), len(val), len(train)

(10000, 10000, 2000000)

In [9]:
def clean_domain_df(df):
    #function clears the domain name from the name of the domain zone
    for i in trange(len(df)):
        while df.loc[i, 'domain']!=re.sub(r'\.[^./]+?$', '', df.loc[i, 'domain']):
            df.loc[i, 'domain']=re.sub(r'\.[^./]+?$', '', df.loc[i, 'domain'])

In [10]:
clean_domain_df(test)
clean_domain_df(val)
clean_domain_df(train)

100%|██████████| 2000000/2000000 [04:06<00:00, 8124.85it/s]


training a model with parameters selected via GridSearchCV

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model = Pipeline([('tfidf', TfidfVectorizer(analyzer='char_wb', ngram_range=(3,6), use_idf=True)),
                              ('nb', MultinomialNB(alpha=0))])

model.fit(train["domain"], train["is_dga"])

pred = model.predict(val["domain"])

writing metrics to a validation.txt

In [ ]:
file = open(path+"validation.txt", "w+")
t_pos=0
f_pos=0
f_neg=0
t_neg=0
#counting the True positive, False positive, False negative, True negative
for i in range(len(val)):
    if val['is_dga'][i]==1:
        if val['is_dga'][i]==pred[i]:t_pos+=1
        else: f_pos+=1
    else:
        if val['is_dga'][i]==pred[i]:t_neg+=1
        else: f_neg+=1
            
file.write('True positive: ' + str(t_pos)+'\n')
file.write ('False positive: ' + str(f_pos)+'\n')
file.write ('False negative: ' + str(f_neg)+'\n')
file.write ('True negative: ' + str(t_neg)+'\n')
file.write ('Accuracy: ' + str(np.mean(pred == val['is_dga']))+'\n')
file.write ('Precision: ' + str(precision_score(val['is_dga'], pred))+'\n')
file.write ('Recall: ' + str(recall_score(val['is_dga'], pred))+'\n')
file.write ('F1: ' + str(f1_score(val['is_dga'], pred))+'\n')
file.close()

In [ ]:
#making a prediction for a test sample
pred=model.predict(test["domain"])

#create a csv file and save it
test.insert (loc= 1 , column='is_dga', value=pred)
test.to_csv('predictions.csv', index=False)

In [ ]:
#save model
joblib.dump(model, 'model.pkl')